In [3]:
import requests
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
import lxml

import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

import pandas as pd

from datetime import datetime

#Отображение колонок и строк в VScode
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [4]:
profs_all = pd.read_csv('profs.txt', header=None, names=['profs_names'])
profs_test = pd.read_csv('profs_test.txt', header=None, names=['profs_names'])

In [5]:
vk_job = pd.DataFrame(columns=['link', 'name', 'location', 'company', 'date', 'description', 'date_of_download'])
df_test = pd.DataFrame(columns=['link', 'name', 'location', 'company', 'date', 'description', 'date_of_download'])

In [6]:
class VkJobParser:

    url = 'https://team.vk.company/vacancy/'
    
    def __init__(self, vac_type):
        '''
        Для инциалиазации задается название вакансии vac_type
        '''
        self.vac_type = vac_type
        self.browser = webdriver.Chrome()
        self.browser.get(self.url)
        self.browser.maximize_window()
        self.browser.delete_all_cookies()
        time.sleep(2)

    def scroll_down_page(self, page_height=0):
        self.browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

        new_page_height = self.browser.execute_script('return document.body.scrollHeight')
        if new_page_height > page_height:
            self.scroll_down_page(new_page_height)
        
    def stop(self):
        '''
        Выйти из Webdriver selenium
        '''
        self.browser.quit()
    
    def find_vacancies(self, profs):
        '''
        Функция для ввода названия вакансии (vac_type) в поисковую строку и клика клавиши Enter
        '''
        for i in range(0, len(profs)):

            top = self.browser.find_element(By.XPATH, '/html/body/div/section/div')
            self.browser.execute_script("return arguments[0].scrollIntoView(true);", top)

            input_button = self.browser.find_element(By.XPATH, '/html/body/div/div[1]/div[1]/div/form/div[1]/div[4]/div/div/div/div/input')

            input_button.send_keys(f"{profs['profs_names'][i]}")
            click_button = self.browser.find_element(By.CLASS_NAME, 'icon-form-search')
            click_button.click()
            time.sleep(5)

            # Прокрутка вниз до конца страницы
            self.scroll_down_page()

            try:
                vacs_bar = self.browser.find_element(By.XPATH, '/html/body/div/div[1]/div[2]/div/div')
                vacs = vacs_bar.find_elements(By.CLASS_NAME, 'result-item')
                vacs = [div for div in vacs if 'result-item' in str(div.get_attribute('class'))]
            
                for vac in vacs:
                    vac_info = {}
                    vac_info['link'] = vac.get_attribute('href')
                    vac_info['name'] = vac.find_element(By.CLASS_NAME, 'title-block').text
                    vac_info['location'] = vac.find_element(By.CLASS_NAME, 'result-item-place').text
                    vac_info['company'] = vac.find_element(By.CLASS_NAME, 'result-item-unit').text
                    vac_info['date_of_download'] = datetime.now()
                    df_test.loc[len(df_test)] = vac_info
                input_button.clear()
            except:
                input_button.clear()
                pass
            
        df_test.drop_duplicates(subset=['link'], keep='last')

    def find_description(self):
        for descr in range(len(df_test)):
            try:
                link = df_test.loc[descr, 'link']
                self.browser.get(link)
                self.browser.delete_all_cookies()
                time.sleep(3)

                df_test.loc[descr, 'description'] = self.browser.find_element(By.CLASS_NAME, 'section').text
            except:
                pass

    def save_df(self):
        df_test.to_csv(f"all_{datetime.now()}.txt", index=False, sep=';')

In [7]:
parser = VkJobParser(profs_test)
parser.find_vacancies(profs_test)
parser.stop()

In [15]:
parser = VkJobParser(profs_all)
parser.find_description()
parser.stop()

In [8]:
df_test

,link,name,location,company,date,description,date_of_download
0,https://team.vk.company/vacancy/33330/,Специалист по оценке персонала,"Москва, комбинированный",VK,NaN,NaN,2023-10-09 16:42:58.091729
1,https://team.vk.company/vacancy/32991/,Специалист по директ-маркетингу,"Москва, комбинированный",Почта,NaN,NaN,2023-10-09 16:42:58.140705
2,https://team.vk.company/vacancy/33556/,Ведущий специалист по закупкам,"Москва, офисный",VK,NaN,NaN,2023-10-09 16:42:58.185679
3,https://team.vk.company/vacancy/8849/,Специалист службы поддержки пользователей,дистанционный,Одноклассники,NaN,NaN,2023-10-09 16:42:58.231683
4,https://team.vk.company/vacancy/33444/,Специалист по модерации в службу поддержки,дистанционный,Одноклассники,NaN,NaN,2023-10-09 16:42:58.276439
5,https://team.vk.company/vacancy/32275/,UI/UX дизайнер,"Санкт-Петербург, гибкий",Одноклассники,NaN,NaN,2023-10-09 16:42:58.322241
6,https://team.vk.company/vacancy/32880/,Специалист по информационной безопасности,"Москва, комбинированный",VK,NaN,NaN,2023-10-09 16:42:58.363219


In [109]:
df_test.to_csv(f"all_vk.txt", index=False, sep='(/)')